# Demonstração

## Importando bibliotecas

In [ ]:
# Data parser: para ler os dados de um arquivo
from util.dataparser import DataParser as dp
# Vizinhos mais próximos: força bruta
from classifiers.neighbors import brute_force_k_neighbors \
    as vizinhos_forca_bruta
# Classificador KNN
from classifiers.neighbors import KNeighborsClassifier
# Para realizar as medidas de impureza
from metrics.measurescores import ImpurityMeasures
# Biblioteca numpy
import numpy as np

## Lendo os dados do arquivo

Utilizando o data parser e um arquivo de teste, podemos fazer a leitura dos dados e converter em um formato mais tradicional para trabalhar com algoritmos de aprendizado de máquina.
A seguir, lemos um arquivo de teste no formato txt.

In [ ]:
ids, X, y = dp.parse('datasets/test_data.txt')
print(X)
print(y)

Cada linha da matriz X representa uma instância do data set e cada coluna representa um atributo e seus valores.
A seguir, iremos ler o data set original da iris obtido pelo arquivo arff do weka.

In [ ]:
ids = None  # Não usaremos a coluna id
X, y = dp.arff_data('datasets/iris.arff')

print(X[:5])  # Imprimindo apenas as primeiras 5 instâncias
print(y[:5])

## Vizinhos mais próximos

O algoritmo __k vizinhos mais próximos__ é um do algoritmos de aprendizado de máquina mais fáceis de implementar. Na verdade, ele é conhecido como um "_lazy algorithm_", ou algoritmo preguiçoso, principalmente pelo fato de não ser necessário uma fase de treinamento para predizer as classes de novas instancias.

Em ```classifiers.neighbors```, implementamos um algoritmo de força bruta para encontrar os k vizinhos mais próximos.

Para predizer a classe da instância, basta fazer uma votação das classes dos vizinhos mais próximos encontrados.

In [ ]:
# Mostra os k vizinhos mais próximos da primeira instância, 
# neste caso, k = 5:
 
distancias, vizinhos = vizinhos_forca_bruta(X, x=X[0], 
                                            n_neighbors=5)

print("Vizinhos mais próximos={}".format(vizinhos))
print("Distâncias calculadas={}".format(distancias))

# Imprime as classes dos vizinhos encontrados
for i in range(len(vizinhos)):
    print(y[i])

É facil perceber que a classe da primeira instância é a _Iris-setosa_

_Nota:todas os métodos da biblioteca utilizam a função de força bruta acima em algum momento._

Implementamos nosso classificador de forma análoga ao classificador da biblioteca sci-kit learn. Por isso, para classificar uma dada instância, podemos instanciar um objeto do KNeighborsClassifier.

In [ ]:
# instancia o classificador knn
classifier = KNeighborsClassifier(algorithm='brute')  

# Como a instância a ser predita está contida no data set de teste, 
# para não computar a própria instância no cálculo, podemos removê-la 
# do data set.

# Transforma em lista:
X = list(X)  
y = list(y)
# Remove o último elemento:
instancia = X.pop()  
classe = y.pop()

classifier.fit(X, y)  # Executa o método fit

# Prediz a instância x (o método predict aceita múltiplas instâncias, 
# por isso x está contido em um vetor):
pred = classifier.predict([instancia])

# Mostra os resultados:
print("Predição={}".format(pred))
print("Real={}".format(classe))

### Vizinhos mais próximos: algoritmo kd-tree

Até agora vimos o algoritmo de classificação executando uma busca em força bruta para encontrar os vizinhos mais próximos, isso porque para cada instância a ser predita, seria calculado a distância com todas as instâncias do data set de teste, e depois selecionado as k instâncias com as menores distâncias.

O sci-kit learn implementa um algoritmo extremamente eficiente para isso chamado kd-tree. A kd-tree é uma arvore onde as folhas agrupam elementos com valores de atributos parecidos, o que melhora muito o desempenho em buscas quando o data set é muito grande.

Tentamos implementar o algoritmo kd-tree em nosso classificador. Apesar de não tão preciso quando o algoritmo do sci-kit learn, consegue predizer muitas instâncias corretamente.

In [ ]:
classifier = KNeighborsClassifier(algorithm='kd_tree')

classifier.fit(X, y)  # Executa o método fit

instancia = X[0]

# Prediz a instância x (o método predict aceita múltiplas instâncias, 
# por isso x está contido em um vetor):
pred = classifier.predict([instancia], ignore_x=True)  

# Mostra os resultados:
print("Predição={}".format(pred))
print("Real={}".format(y[0]))

## Medidas de impureza

Estamos interessados em medir as impurezas dos dados. Medir impurezas é extremamente útil, principalmente, por exemplo, se estivéssemos interessados em obter árvores de decisão para classificar novas instâncias.

Implementamos as medidas gini, entropia e o erro de classificação na biblioteca measurescores.

Após instanciar um objeto de ```ImpurityMeasures``` com os nossos dados, podemos calcular as medidas de impureza de um dado atributo.

In [ ]:
# Instancia um objeto de ImpurityMeasures
medidas = ImpurityMeasures(X, y)  

# Iremos calcular para cada atributo(índice da coluna)
for atributo in range(len(X[0])):  
    print("Para o atributo {}".format(atributo)) 
    print("Porcentagens={}".format(medidas.percentages(atributo)[:10]))
    print("Ginis={}".format(medidas.gini(atributo)[:10]))
    print("Entropias={}".format(medidas.entropy(atributo)[:10]))
    print("Erros de classificação={}".format(medidas.
                                             classification_error(atributo)[:10]))

# Imprime a medida gini para o primeiro valor do primeiro atributo:
print("Medida gini para o primeiro valor do primeiro atributo:")
print(medidas.gini(0)[0])

Observe que para cada calculo de medida, todos os dados de impureza de cada instancia e cada valor de atributo são retornados (note que existem muitos valores repetidos, porque existem valores de atributos iguais para uma dada classe, isso implica que a impureza seja igual nesses casos).

Talvez todos esses dados aparentem serem confusos, mas observe o retorno da função na documentação do método ```percentages``` por exemplo:

```python
"""
Compute all the percentages p(yi) of a given feature.

:param feature: attribute index to compute the percentage
:return: numpy array shape [attribute values, class labels]
"""
```

O retorno da função é uma matriz onde cada linha representa o respectivo valor do atributo, e cada coluna a respectiva classe. Logo, o valor Pij da matriz representa a porcentagem P do valor do atributo na instancia i, com relação a classe j (note que a soma de cada linha é igual a 1!)

Esse método é extremamente útil porque é utilizado para calcular todas as medidas de impureza.

Já para a medida gini, a documentação fornece:

```python
"""
Compute the gini measures of a given feature in the data set.

:param feature: feature to compute
:return: numpy array shape [attribute values]
"""
```

Isto é, para cada valor de atributo de um dado atributo(coluna do data set), retornará um vetor contendo as medidas gini de cada valor (as demais medidas de impureza funcionam de forma análoga).

Se quisessemos dividir o data set em faixas o mais puras possível, escolheríamos os valores de impureza menores possíveis e separaríamos as instâncias com valores menores e maiores que o escolhido.

Observe para um data set menor os dados retornados:

In [ ]:
X = np.asarray([[1, 1, 0],  # classe 1
                [1, 0, 1],  # classe 2
                [1, 0, 1],  # classe 2
                [0, 1, 0],  # classe 1
                [0, 1, 1]]) # classe 2 
y = ['classe 1', 'classe 2', 'classe 2', 'classe 1', 'classe 2']

medidas = ImpurityMeasures(X, y)

# Iremos calcular para cada atributo(índice da coluna):
for atributo in range(len(X[0])):  
    print("Para o atributo {}".format(atributo)) 
    print("Porcentagens={}".format(medidas.percentages(atributo)))
    print("Ginis={}".format(medidas.gini(atributo)))
    print("Entropias={}".format(medidas.entropy(atributo)))
    print("Erros de classificação={}".format(medidas.
                                             classification_error(atributo)))

Observe os valores das medidas de impureza do último atributo, todos os valores do atributo dividem perfeitamente o data set nas duas classes que temos, isso implica a maior pureza possível, exatamente como obtido nos cálculos das medidas.